In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import BooleanType
import math
#from rateFunction import *
spark = SparkSession.builder.appName("Ngram_buzzwords").master("spark://192.168.1.76:7077").getOrCreate()
sc = spark.sparkContext

In [ ]:
def sortBuzzWords(buzzwords, index_value):
    return sorted(buzzwords, key=lambda x: x[index_value], reverse=True)

In [ ]:
df = spark.read.csv('../../data/eng-gb-all-only-letters.csv', sep="\t", header=False)

In [ ]:
df = df.withColumnRenamed('_c0', 'ngram')
df = df.withColumnRenamed('_c1', 'year')
df = df.withColumnRenamed('_c2', 'match_count')
df = df.withColumnRenamed('_c3', 'page_count')
df = df.withColumnRenamed('_c4', 'volume_count')

In [ ]:
totalCountsDf = spark.read.csv('../../data/eng-gb-all/googlebooks-eng-gb-all-totalcounts-20090715.txt', sep="\t", header=False)

In [ ]:
totalCountsDf = totalCountsDf.withColumnRenamed('_c0', 'year')
totalCountsDf = totalCountsDf.withColumnRenamed('_c1', 'match_count')
totalCountsDf = totalCountsDf.withColumnRenamed('_c2', 'page_count')
totalCountsDf = totalCountsDf.withColumnRenamed('_c3', 'volume_count')

In [ ]:
totalCounts = sc.broadcast(totalCountsDf.collect())

In [ ]:
totalCountsRDD = totalCountsDf.rdd.map(list)

In [ ]:
totalCounts = sc.broadcast(dict(totalCountsRDD.map(lambda x: (x[0], (x[1],x[2],x[3]))).collect()))

In [ ]:
totalCounts.value["1944"]

In [ ]:
totalCounts.value

In [ ]:
#df.count()

In [ ]:
rdd = df.rdd.map(list)

In [ ]:
def splitAndMakeNgramKey(row):
    elements = row[0].split(",")
    return(elements[0], (elements[1:len(elements)]))

rdd = rdd.map(splitAndMakeNgramKey)

In [ ]:
#rdd.take(10)

In [ ]:
#rdd2 = sc.parallelize(rdd.take(1000000))

In [ ]:
def groupData(a,b):
    if isinstance(a[0], (list,)):
        return a+[b]
    else:
        return [a]+[b]

rdd2 = rdd.reduceByKey(groupData)

In [ ]:
def yearRate(word):
    if len(word) == 0:
        return [('0',0,0.0,0,0,0.0)]
    
    if len(word) == 1:
        return [(word[0][0], 0, 0.0,0,0,0.0)]
    
    wordArray=[]
    for i in range(len(word)-1):
        try:
            yearPair = str(word[i][0])+"-"+str(word[i+1][0])
            diff = int(word[i+1][1])-int(word[i][1])
            diffProcent = (float(word[i+1][1])/float(word[i][1]))*100
            ratio=diffProcent/100
            buzzWordValue = (float(diff)/math.sqrt(float(totalCounts.value[word[i+1][0]][0])))*ratio
            #buzzWordValue = diff*diffProcent
            wordArray.append((yearPair,diff,diffProcent,int(word[i][1]),int(word[i+1][1]),buzzWordValue))
        except:
            return [('0',0,0.0,0,0,0.0)]
    return wordArray

In [ ]:
def yearRateThresh(word,t):
    if len(word) == 0:
        return [('0',0,0.0,0,0,0.0)]
    
    if len(word) == 1:
        return [(word[0][0], 0, 0.0,0,0,0.0)]
    
    wordArray=[]
    for i in range(len(word)-1):
        try:
            if int(word[i][0])>t:
                yearPair = str(word[i][0])+"-"+str(word[i+1][0])
                diff = int(word[i+1][1])-int(word[i][1])
                diffProcent = (float(word[i+1][1])/float(word[i][1]))*100
                ratio=diffProcent/100
                buzzWordValue = (float(diff)/math.sqrt(float(totalCounts.value[word[i+1][0]][0])))*ratio
            #buzzWordValue = diff*diffProcent
                wordArray.append((yearPair,diff,diffProcent,int(word[i][1]),int(word[i+1][1]),buzzWordValue))
            else: 
                wordArray.append('0',0,0.0,0,0,0.0)
        except:
            return [('0',0,0.0,0,0,0.0)]
    return wordArray

In [ ]:
rdd3 = rdd2.map(lambda row: (row[0], yearRate(row[1])))

In [ ]:
rdd4 = rdd2.map(lambda row: (row[0], yearRateThresh(row[1],1980)))

In [ ]:
#rdd_count = rdd3.map(lambda row: (row[0], sortBuzzWords(row[1], 1)))

In [ ]:
rdd_percent = rdd3.map(lambda row: (row[0], sortBuzzWords(row[1], 2)))

In [ ]:
#rdd_buzzword = rdd3.map(lambda row: (row[0], sortBuzzWords(row[1], 5)))
rdd_buzzword = rdd4.map(lambda row: (row[0], sortBuzzWords(row[1], 5)))

In [ ]:
#rdd_count2 = rdd_count.sortBy(lambda row: row[1][0][1], ascending=False)

In [ ]:
rdd_percent2 = rdd_percent.sortBy(lambda row: row[1][0][2], ascending=False)

In [ ]:
rdd_buzzword2 = rdd_buzzword.sortBy(lambda row: row[1][0][5], ascending=False)

In [ ]:
#rdd_count2.take(10)

In [ ]:
rdd_percent2.take(10)

In [ ]:
rdd_buzzword2.take(10)

In [ ]:
#rdd3.coalesce(1).saveAsTextFile("Words-with-year-list.txt")